In [6]:
print("Hello Notebook Mode!")

Hello Notebook Mode!


In [7]:
import json
import os
import sys
import logging
from typing import Optional, List, TypedDict

# Add parent directory to Python path to import backend modules
backend_dir = os.path.dirname(os.path.dirname(os.path.abspath('.')))
if backend_dir not in sys.path:
    sys.path.insert(0, backend_dir)

from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI
from azure.core.exceptions import AzureError
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential

# Load environment variables
load_dotenv()

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
# Search query
message = "what is BCID"

# Get Azure Search credentials from environment
search_endpoint = os.environ.get("AZURE_SEARCH_ENDPOINT")
search_key = os.environ.get("AZURE_SEARCH_KEY")
index_name = os.environ.get("AZURE_SEARCH_INDEX_NAME")

# Verify credentials are loaded
print("Checking Azure Search Configuration...")
if not all([search_endpoint, search_key, index_name]):
    print("Missing Azure Search credentials!")
else:
    
    try:
        # Create search client
        credential = AzureKeyCredential(search_key)
        client = SearchClient(
            endpoint=search_endpoint,
            index_name=index_name,
            credential=credential,
        )
        
        # Perform search
        print(f"\nSearching for: '{message}'")
        search_results = client.search(
            search_text=message,
            select=["*"],
            top=5,
        )
        
        # Convert and display results
        results = []
        for i, result in enumerate(search_results, 1):
            result_dict = dict(result)
            results.append(result_dict)
            
            print(f"\nResult {i}:")
            print(f"  Score: {result_dict.get('@search.score', 'N/A')}")
            
            # Display key fields (adjust based on your index schema)
            for key, value in result_dict.items():
                if not key.startswith('@') and value:
                    str_value = str(value)
                    # Truncate long values for readability
                    if len(str_value) > 150:
                        str_value = str_value[:150] + "..."
                    print(f"  {key}: {str_value}")
        
        print(f"\n✓ Search completed - Found {len(results)} results")
        
    except AzureError as e:
        logger.error(f"Azure error: {e}", exc_info=True)
    except Exception as e:
        logger.error(f"Error during search: {e}", exc_info=True)

INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://css-ai-dev-caddy.azurewebsites.net/search/indexes('bc-water-index')/docs/search.post.search?api-version=REDACTED'
Request method: 'POST'
Request headers:
    'Content-Type': 'application/json'
    'Content-Length': '51'
    'api-key': 'REDACTED'
    'Accept': 'application/json;odata.metadata=none'
    'x-ms-client-request-id': '3180ad44-cc98-11f0-ad37-86df7cf5b419'
    'User-Agent': 'azsdk-python-search-documents/11.6.0 Python/3.14.0 (macOS-15.3.2-arm64-arm-64bit-Mach-O)'
A body is sent with the request


Checking Azure Search Configuration...

Searching for: 'what is BCID'


INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 200
Response headers:
    'Content-Type': 'application/json; odata.metadata=none; odata.streaming=true; charset=utf-8'
    'Date': 'Fri, 28 Nov 2025 20:24:09 GMT'
    'Transfer-Encoding': 'chunked'
    'Via': 'REDACTED'
    'Strict-Transport-Security': 'REDACTED'
    'Elapsed-Time': 'REDACTED'
    'Odata-Version': 'REDACTED'
    'Preference-Applied': 'REDACTED'
    'Request-Id': '3180ad44-cc98-11f0-ad37-86df7cf5b419'



Result 1:
  Score: 5.244277
  id: ZGU0YTMzMGItOTVmZC00ZDQ3LThmYWQtNzZmMWY1YmI2MDAw
  metadata: {"source": "water_use_purpose_defns.pdf", "processed_with": "azure_document_intelligence"}
  content: "Step 2 - Please select one of the following purpose uses?" These are the water use subpurposes in blue text below (beginning on page 2)
Please note t...

Result 2:
  Score: 5.195652
  id: NWE2Yzc2MTUtZWNmMC00NjkzLTg5ZjMtNGVhZDQ1YmFmYmUy
  metadata: {"source": "water_use_purpose_defns.pdf", "processed_with": "azure_document_intelligence"}
  content: "Step 2 - Please select one of the following purpose uses?" These are the water use subpurposes in blue text below (beginning on page 2)
Please note t...

Result 3:
  Score: 5.166315
  id: NTM1MWZkNjMtZWQxMy00MzM0LWJkYjUtZTY3YTkxNzEwMmYx
  metadata: {"source": "water_use_purpose_defns.pdf", "processed_with": "azure_document_intelligence"}
  content: "Step 2 - Please select one of the following purpose uses?" These are the water use subpurposes in